# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import json

### 先搜尋全部的電影代號(ID)資訊

In [2]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))
    
    


Movie: 航海王：奪寶爭霸戰, ID: 10169
Movie: 全面攻佔3：天使救援, ID: 10082
Movie: 乾杯！戀上日本酒的女子, ID: 10131
Movie: 普羅米亞, ID: 10119
Movie: 怨靈少女組, ID: 10103
Movie: 驅魔使者, ID: 10095
Movie: 炫目之光, ID: 10066
Movie: 飛翔吧！埼玉, ID: 9833
Movie: 緝魔, ID: 9795
Movie: 憤怒鳥玩電影2：冰的啦, ID: 10002
Movie: 櫻花開了, ID: 10184
Movie: 乘浪之約, ID: 10168
Movie: 出發吧！我的脫單假期, ID: 10151
Movie: 懸案判決, ID: 10142
Movie: 怎麼就，結婚了, ID: 10133
Movie: 絕世名伶, ID: 10132
Movie: 是誰在搞鬼, ID: 10102
Movie: 蠟筆小新：新婚旅行風暴─奪回廣志大作戰, ID: 10078
Movie: 一定要結婚嗎, ID: 10074
Movie: 朵拉與失落的黃金城, ID: 9920
Movie: 在黑暗中說的鬼故事, ID: 10063
Movie: 靈異乍現, ID: 9456
Movie: 瑪雅蜜蜂大冒險：蜜糖危機, ID: 10139
Movie: 痛苦與榮耀, ID: 10126
Movie: 王的文字, ID: 10118
Movie: 電梯男孩的奇蹟, ID: 10114
Movie: 玩命關頭：特別行動, ID: 9582
Movie: 出走巴黎, ID: 10140
Movie: 鬼學怨, ID: 10135
Movie: 駭客任務, ID: 10088
Movie: 人生消極掰, ID: 10059
Movie: 電影哆啦A夢：大雄的月球探測記, ID: 10016
Movie: 從前，有個好萊塢, ID: 9884
Movie: 王者天下, ID: 10125
Movie: 首席指揮家, ID: 10116
Movie: 電流大戰, ID: 10021
Movie: 獅子王, ID: 9116
Movie: 名偵探柯南：紺青之拳, ID: 9635
Movie: 米開朗基羅：無盡之詩, ID: 10100
Movi

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [9]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10413

In [144]:
import requests
import re
from bs4 import BeautifulSoup
import json

movie_id = 10413
url = 'https://movies.yahoo.com.tw/movietime_result.html/id=10413'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼
soup = BeautifulSoup(resp.text,"html5lib")
#print(soup)
area = soup.find("select",attrs={"class":"moviearea"})
print(area)
for a in area:
   # print(a)
    #print(a.string)
    print(a("select")["value"])
    
# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
#for p in resp.json() :
 #   print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

<select class="moviearea timelist_area" name="area_id">
                <option value="">選擇地點</option>
                                    <option value="28">台北市</option>
                                    <option value="3">台北二輪</option>
                                    <option value="8">新北市</option>
                                    <option value="16">桃園</option>
                                    <option value="20">新竹</option>
                                    <option value="15">苗栗</option>
                                    <option value="2">台中</option>
                                    <option value="22">彰化</option>
                                    <option value="13">南投</option>
                                    <option value="19">雲林</option>
                                    <option value="21">嘉義</option>
                                    <option value="10">台南</option>
                                    <option value="17">高雄</option>
                         

TypeError: 'NavigableString' object is not callable

### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [11]:
# 指定放映地區
area_id = 28

In [12]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

八月 21
八月 22
八月 23
八月 24
八月 25


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [70]:
# 選定要觀看的日期
date = "2020-12-20"

In [89]:
# 向網站發送請求，獲取上映的電影院及時間資訊
movie_id = 10413
area_id = 28
date = "2020-12-20"

url = "https://movies.yahoo.com.tw/movietime_result.html?movie_id=10413&area_id=28"
"""payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}"""

resp = requests.get(url) #params=payload
soup = BeautifulSoup(resp.text,"html5lib")
print(soup)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

#soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})
#for h in html:
#print(html)

<!DOCTYPE html>
<html lang="zh_TW"><head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, user-minimum-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="501887343352051" property="fb:app_id"/>
  <meta content="Yahoo奇摩電影" property="og:site_name"/>
    <title>神力女超人1984 Wonder Woman 1984電影時刻表 - Yahoo奇摩電影</title>
<meta content="神力女超人1984 Wonder Woman 1984電影時刻表 - Yahoo奇摩電影" name="twitter:title"/>
  <meta content="神力女超人1984 Wonder Woman 1984電影時刻表 - Yahoo奇摩電影" property="og:title"/>

<link href="https://movies.yahoo.com.tw/movietime_result.html?movie_id=10413&amp;area_id=28" rel="canonical"/>
<meta content="https://movies.yahoo.com.tw/movietime_result.html?movie_id=10413&amp;area_id=28" property="og:url"/>

<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" name="twitter:description"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" name="descript

In [96]:
import requests
import re
from bs4 import BeautifulSoup
import json

url = "https://movies.yahoo.com.tw/movietime_result.html?movie_id=10413&area_id=29"
"""payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}"""

resp = requests.get(url) #params=payload
soup = BeautifulSoup(resp.text,"html5lib")
print(soup)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

#soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})
#for h in html:
#print(html)

<!DOCTYPE html>
<html lang="zh_TW"><head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, user-minimum-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="501887343352051" property="fb:app_id"/>
  <meta content="Yahoo奇摩電影" property="og:site_name"/>
    <title>Yahoo奇摩電影</title>
<meta content="Yahoo奇摩電影" name="twitter:title"/>
  <meta content="Yahoo奇摩電影" property="og:title"/>

<link href="https://movies.yahoo.com.tw?err=404&amp;err_url=http%3A%2F%2Fmovies.yahoo.com.tw%2Fmovietime_result.html&amp;err_message=area_id%3A+29+not+found&amp;sign=acfac1da48f3a857ebd574641d738fde510d98c6" rel="canonical"/>
<meta content="https://movies.yahoo.com.tw?err=404&amp;err_url=http%3A%2F%2Fmovies.yahoo.com.tw%2Fmovietime_result.html&amp;err_message=area_id%3A+29+not+found&amp;sign=acfac1da48f3a857ebd574641d738fde510d98c6" property="og:url"/>

<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿

In [16]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''


----------------------------------------------------------------------
電影院:  台北美麗華大直影城
放映類型:  數位
2019-08-21 09:00:00
2019-08-21 11:10:00
2019-08-21 13:15:00
2019-08-21 15:20:00
2019-08-21 19:30:00
2019-08-21 21:40:00
2019-08-21 22:30:00
----------------------------------------------------------------------
電影院:  台北新光影城
放映類型:  數位
2019-08-21 10:00:00
2019-08-21 14:50:00
2019-08-21 19:30:00
----------------------------------------------------------------------
電影院:  台北in89豪華數位影城
放映類型:  數位
2019-08-21 09:30:00
2019-08-21 11:20:00
2019-08-21 13:15:00
2019-08-21 15:10:00
2019-08-21 16:10:00
2019-08-21 17:10:00
2019-08-21 18:05:00
2019-08-21 19:10:00
2019-08-21 21:10:00
2019-08-21 23:10:00
2019-08-22 01:10:00
----------------------------------------------------------------------
電影院:  台北日新威秀影城
放映類型:  數位
2019-08-21 09:00:00
2019-08-21 10:55:00
2019-08-21 12:50:00
2019-08-21 14:45:00
2019-08-21 16:40:00
2019-08-21 18:35:00
2019-08-21 20:35:00
-----------------------------------------------------

In [42]:
#練習
import requests
import re 
from bs4 import BeautifulSoup 

url = "https://movies.yahoo.com.tw/movie_intheaters.html"
r = requests.get(url)
response = r.text 
soup = BeautifulSoup(response,"html5lib")
print(soup)
html = soup.find("div",attrs={"class":"release_box"})
print(html.p.string)

<!DOCTYPE html>
<html lang="zh_TW"><head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, user-minimum-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="501887343352051" property="fb:app_id"/>
  <meta content="Yahoo奇摩電影" property="og:site_name"/>
  
  <title>上映中 - Yahoo奇摩電影</title>
<meta content="上映中 - Yahoo奇摩電影" name="twitter:title"/>
  <meta content="上映中 - Yahoo奇摩電影" property="og:title"/>

<link href="https://movies.yahoo.com.tw/movie_intheaters.html" rel="canonical"/>
<meta content="https://movies.yahoo.com.tw/movie_intheaters.html" property="og:url"/>

<meta content="Yahoo奇摩電影提供上映中的最新電影資訊，查詢時刻表、看最新預告片，讓你電影資訊不漏接" name="twitter:description"/>
<meta content="Yahoo奇摩電影提供上映中的最新電影資訊，查詢時刻表、看最新預告片，讓你電影資訊不漏接" name="description"/>

  <meta content="Yahoo奇摩電影提供上映中的最新電影資訊，查詢時刻表、看最新預告片，讓你電影資訊不漏接" property="og:description"/>

<meta content="https://movies.yahoo.com.tw/build/images/og

In [68]:
#練習
max_page = 8 
for page_number in range(1,max_page+1):
    url = "https://movies.yahoo.com.tw/movie_intheaters.html"
    payload = {"page":str(page_number)} #辭典
    resp = requests.get(url,params=payload)
    resp.encoding = "utf-8"
    soup = BeautifulSoup(resp.text,"lxml")
    
    movie_list = soup.find("ul",attrs={"class":"release_list"}).find_all("li")
    for p in movie_list:
        print("---------------------------------------")
        #print(p)
        #電影名稱
        movie_name = p.find("div",attrs={"class":"release_movie_name"})
        print("電影名稱",movie_name.a.string)
        #print("電影名稱",movie_name)
        
        #定位電影評價資訊欄位
        level_box = movie_name.find("dl",attrs={"class":"levelbox"})
        #print(level_box)
        
        #期待度
        expectation = level_box.find("div", attrs={"class":"leveltext"})
        print("期待度：",expectation.span.string)
  
        #滿意度
        satisfaction = level_box.find("span",attrs={"class":"count"})
        print("滿意度：",satisfaction["data-num"])
        
        #簡介
        movie_info = p.find("div",attrs={"class":"release_text"})
        print(movie_info.span.string)
        
        
    
    

---------------------------------------
電影名稱 
                  求婚好意外
期待度： 88%
滿意度： 4.3

                  艾比(克莉絲汀史都華 飾演)原本打算趁聖誕假期在家族聚餐上向女友哈珀(麥坎西戴維斯 飾演)求婚，沒想到哈珀壓根還沒跟保守的家人出櫃。關係曝光讓原本歡樂的假期陷入一團混亂，這對戀人該如何面對自己和親友勇敢說愛？同時又不毀掉完美的聖誕假期？
                
---------------------------------------
電影名稱 
                  怵目驚魂28天
期待度： 64%
滿意度： 4.0

                  開啟燒腦話題，全球推崇獨立經典，傑克葛倫霍影史最具代表性作品。
 
資質聰穎的唐尼想法異於同儕，一直以來都被解讀為怪胎，被學校認為是問題學生，被家人認為精神狀態異常，因此之前闖下的禍而必須接受心理醫生諮詢。
 
一天唐尼接受到奇幻人物的訊息，警告他世界末日即將在28天後到來。當天晚上，一架飛機的引擎掉落進東尼家的屋頂，唐尼因為奇幻人物的召喚逃過一劫。
 
看似平靜的校園裡，人與人之間充滿不了解與衝突，隱藏著未知的危機。隨著倒數的時間逐漸接近，唐尼陸續做出毀滅性的脫軌行為。末日來臨前的最後一刻，唐尼突然驚覺，他是唯一可以扭轉世界命運的人。
                
---------------------------------------
電影名稱 
                  信用詐欺師JP：公主篇
期待度： 95%
滿意度： 4.4

                  ★峰迴路轉燒腦歡快，日本賣座神片續集，原班人馬重磅出擊！
★ 蟬聯五週日本票房排行榜！超越前作大賣40億日幣!
★「眼睛看到的未必是真實。什麼是真，什麼是假。你能看穿真實嗎！？歡迎來到詐欺師的世界。」
★電影主題曲-Laughter【Official 鬚男dism】深情獻唱
 
《公主篇》以馬來西亞「蘭卡威島」作為舞台背景，世上頂尖大富豪-胡.雷盟身後留下10兆日圓遺產，律師發表的繼承人卻是誰都不知道的私生女-蜜雪兒。就在家屬尋找私生女的同時，世界各地的詐欺師們也為這遺產大集合！詐欺師三

---------------------------------------
電影名稱 
                  黑暗角落：小丑夢靨
期待度： 0
滿意度： 0.0

                  大多數的人在小時候對夜晚都有著無限的恐懼，睡在床上想像著窗外有個很可怕的生物會把突然闖入把自己從床上抓走，《黑暗角落：小丑夢靨》將恐懼變成了現實，進入到殺手小丑四處遊蕩的世界，睡夢中不斷被小丑給嚇醒，也不確定到底是夢境還是現實，你真的可以逃離小丑世界嗎？
 
黑暗角落：小丑夢靨+黑暗角落：死亡之後  兩部電影接續放映
 
原創VR電影  就在 國賓影城＠台北長春廣場VR劇院
11/27起 開放現場&網路購票
上映資訊：https://viveoriginals.com/vrcinema2/
訂票專屬網站：https://vrcinema.tixi.com.tw/
 
                
---------------------------------------
電影名稱 
                  黑暗角落：死亡之後
期待度： 100%
滿意度： 0.0

                  急救無效宣布死亡之際，人還能感受到周圍的事物嗎? 《黑暗角落：死亡之後》引領觀眾體驗人生最後時刻，並嘗試拼湊出自己是誰，以及怎麼來到這裡。但緊接下來發生的一切，卻比死亡本身更為驚悚……
※ 黑暗角落：小丑夢靨+黑暗角落：死亡之後  兩部電影接續放映
 
原創VR電影  就在 國賓影城＠台北長春廣場VR劇院
11/27起 開放現場&網路購票
上映資訊：https://viveoriginals.com/vrcinema2/
訂票專屬網站：https://vrcinema.tixi.com.tw/
 
                
---------------------------------------
電影名稱 
                  藝術文明之光：夏維岩洞
期待度： 0
滿意度： 0.0

                  位在法國阿爾代什( Ardèche)的夏維岩洞(Chauvet Cave)，洞內岩壁上有上千幅可追溯至距今3萬6千年前的畫作史前壁畫，其中描繪動物連續性動作的動態圖畫，如同最早的動畫技術，1

---------------------------------------
電影名稱 
                  家在蘭若寺
期待度： 50%
滿意度： 2.2

                  ★第74屆威尼斯影展競賽單元
★2019上海國際電影節
★2019日內瓦國際影展
★2018墨爾本國際電影節  
★2017台北金馬國際影展
★第42屆香港國際電影節  
 
蔡明亮執導的全球首部華語VR電影長片。
入圍2017年第74屆威尼斯影展VR競賽單元，作品安排在小島Lazzaretto Vecchio上播映，乃威尼斯影展為VR播映量身訂作擁有50個位置的16世紀建築劇院中觀賞作品。 
 
故事描述小康在山裡養病，他的亡母來為他做飯，但是他吃不到。
他的鄰居是一名女鬼，也走不進他的生活，他唯一傾訴的對象-是一條魚。
 
【售票資訊】
 
10/30 (五) 開放預售
購票請上 https://vrcinema.tixi.com.tw/
 
                
---------------------------------------
電影名稱 
                  鬼滅之刃劇場版 無限列車篇
期待度： 97%
滿意度： 4.5

                  ★全球觀眾引僅期盼，最〝燃〞神作動畫首部劇場版 
★《獵人》作者「富堅義博」感動力推 
★日本漫畫總銷量 5 年內突破 6 千萬
 
《鬼滅之刃》是家人慘遭鬼殺害的少年－竈門炭治郎，為了讓化為鬼的妹妹禰豆子恢復回人 類，自願加入「鬼殺隊」的故事。以人鬼間的悲痛故事、驚心動魄的劍戰，以及偶然穿插的 滑稽場景，贏得廣大人氣，不僅紅遍日本，更掀起全球觀眾的熱烈討論。 
 
緊接在電視版動畫《竈門炭治郎‧立志篇》之後的故事《無限列車篇》，即將登上大銀幕。 炭治郎等人完成「蝴蝶屋」的訓練，下一個目的地是開往黑暗的「無限列車」。 
 
以多人行蹤不明的這輛列車為舞台，炭治郎帶著禰豆子與善逸、伊之助一行人，與鬼殺隊最 強劍士〝柱〞其中之一「炎柱‧煉獄杏壽郎」會合， 新的任務即將開始！ 
 
                
---------------------------------------
電影名稱 
                  玩美女人 經典數

---------------------------------------
電影名稱 
                  消失的情人節
期待度： 81%
滿意度： 4.2

                  公車司機阿泰（劉冠廷/飾），是一個動作慢吞吞、連手錶的時間、地震的感應都比別人慢的奇葩。他每天都會去郵局，找櫃員楊曉淇（李霈瑜/飾）寄一封不知道給誰的平信。櫃員楊曉淇和阿泰完全相反，是個凡事搶拍的超級急性子，唯獨感情生活毫無進展，年近三十還是母胎單身。情人節前夕，楊曉淇被天菜陽光男劉老師（周群達Duncan/飾）搭訕，終於情竇初開的曉淇和劉老師天雷勾動地火；想不到一覺醒來，劉老師再無音訊、而且情人節居然已經過了？！每天來寄信的阿泰也再也沒出現過...毫無記憶的楊曉淇，開始了一趟尋找真相的奇幻旅程...
                
---------------------------------------
電影名稱 
                  寂寞調香師
期待度： 67%
滿意度： 0.0

                  世間所有相遇　都能香氣四溢
熟男司機吉翁（格雷戈里蒙泰爾 飾），只有一件事掛心不下，就是爭取十歲女兒的共同監護權。然而他的違規計點已到吊銷邊緣，難得的父女時光也常因自己的粗枝大葉搞砸。吉翁的老闆清楚他的「魯」，特意安插他接送最棘手的高端客戶安（艾曼紐德芙 飾）。擁有絕對嗅覺的安，曾是Dior天價聘請的調香大師。在她人生最輝煌的時刻，卻意外嗅覺缺失，從香水界黯然退場，勉強委身在五味雜陳的案子沉潛自己。
 
一位是退無可退的魯蛇司機，一位是渴望重返巔峰的調香達人，兩人天差地遠，在生命轉折處撞見彼此，即將在一趟趟採集氣味的旅程中，釋放出動人的曖昧氣息。
 
                
---------------------------------------
電影名稱 
                  皮諾丘的奇幻旅程
期待度： 86%
滿意度： 3.9

                  皮諾丘能通過考驗，如願成為一個真正的男孩嗎？
孤獨的老木匠傑佩托，因緣際會得到一塊櫻桃木，便將它雕刻成一個木偶，並取名「皮諾丘」來作伴。意外的是，木偶完成後不僅能開口說話，更會走能跑。傑佩托喜出望外，決定好好栽培，於